In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
cifar100 = keras.datasets.cifar100

(x_train,y_train),(x_test,y_test) = cifar100.load_data()
x_train,x_test = x_train/255.0, x_test/255.0

In [3]:
model = keras.Sequential([
    keras.layers.Conv2D(16,3,activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32,3,activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256,activation='relu'),
    keras.layers.Dense(100,activation='softmax')
])

In [4]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
model.fit(x_train,y_train,epochs=1)
model.evaluate(x_test,y_test,verbose=1)

2023-01-09 11:48:07.668247: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 11:48:07.668998: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.


Train on 50000 samples
50000/50000 [==============================] - 46s 918us/sample - loss: 3.6379 - accuracy: 0.1524
10000/1 [======================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[3.147088341522217, 0.2391]

# 23-7. Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (2) Functional API 활용

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [5]:
cifar100 = keras.datasets.cifar100
(x_train,y_train),(x_test,y_test) = cifar100.load_data()
x_train,x_test = x_train/255.0, x_test/255.0

In [8]:
inputs = keras.Input(shape=(32,32,3))
x = keras.layers.Conv2D(16,3,activation='relu')(inputs)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32,3,activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256,activation='relu')(x)
predictions = keras.layers.Dense(100,activation='softmax')(x)

model = keras.Model(inputs=inputs,outputs=predictions)

In [10]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])
model.fit(x_train,y_train,epochs=1)
model.evaluate(x_test,y_test,verbose=2)

Train on 50000 samples
50000/50000 [==============================] - 44s 883us/sample - loss: 3.6174 - accuracy: 0.1558
10000/1 - 5s - loss: 3.1507 - accuracy: 0.2375


[3.172152751159668, 0.2375]

# 23-8. Tensorflow2 API로 모델 작성 및 학습하기: CIFAR-100 (3) Subclassing 활용

In [11]:
import tensorflow as tf
from tensorflow import keras

In [14]:
cifar100 = keras.datasets.cifar100
(x_train,y_train),(x_test,y_test) = cifar100.load_data()
x_train,x_test = x_train/255.0, x_test/255.0

In [16]:
class MyModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16,3,activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32,3,activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.dense1 = keras.layers.Dense(256, activation='relu')
        self.dense2 = keras.layers.Dense(100,activation='softmax')
        
    def call(self,x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x
        
        
model = MyModel()

In [17]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.fit(x_train,y_train,epochs=1)
model.evaluate(x_test,y_test,verbose=2)

Train on 50000 samples
50000/50000 [==============================] - 46s 929us/sample - loss: 3.6374 - accuracy: 0.1532
10000/1 - 5s - loss: 3.2231 - accuracy: 0.2340


[3.180981675720215, 0.234]

# 23-9. GradientTape의 활용

In [18]:
import tensorflow as tf
from tensorflow import keras

cifar100 = keras.datasets.cifar100

(x_train,y_train),(x_test,y_test) = cifar100.load_data()
x_train,x_test = x_train/255.0, x_test/255.0

class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16,3,activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32,3,activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.dense1 = keras.layers.Dense(256, activation='relu')
        self.dense2 = keras.layers.Dense(100,activation='softmax')
        
    def call(self,x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x
        
        
model = CustomModel()

In [19]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

def train_step(features,labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels,predictions)
        gradients = tape.gradient(loss,model.trainable_variables)
        
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
    result = zip(gradients,model.trainable_variables)
    print("zip result : ",result)
    return loss

In [ ]:
model.fit(x_train,y_train,epochs=2,batch_size=32)

In [21]:
import time
import numpy as np
def train_model(batch_size=32):
    start = time.time()
    for epoch in range(1): #전체 에포크 5번 반복 
        x_batch = []
        y_batch = []
        for step, (x,y) in enumerate(zip(x_train,y_train)):
            x_batch.append(x)
            y_batch.append(y)
            if step % batch_size == batch_size-1:
                loss = train_step(np.array(x_batch,dtype=np.float32),np.array(y_batch, dtype=np.float32))
                x_batch=[]
                y_batch=[]
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))
train_model()

Epoch 0: last batch loss = 3.0996
It took 238.798926115036 seconds
